In [60]:
from mesh_to_sdf import mesh_to_sdf

import trimesh
import pyrender
import numpy as np


In [64]:
tfs = np.eye(4)
# First is small translation to get the anchor to front left corner
# Then translate anchor to desired location
tfs[:3,3] = np.array([5,2.5,-2.5]) + np.array([0, 0, 10])
# tfs[:3,3] = np.array([-40.0, -30.0, -2.5]) + np.array([0, 0, 10])

sm = trimesh.creation.box(np.array([10.0, 5.0, 5.0]), transform=tfs)
sm.visual.vertex_colors = [0.0, 1.0, 0.0, 0.1]

sm.export("./custom_box.obj")



'# https://github.com/mikedh/trimesh\nv 0.00000000 0.00000000 5.00000000 0.00000000 1.00000000 0.00000000\nv 0.00000000 0.00000000 10.00000000 0.00000000 1.00000000 0.00000000\nv 0.00000000 5.00000000 5.00000000 0.00000000 1.00000000 0.00000000\nv 0.00000000 5.00000000 10.00000000 0.00000000 1.00000000 0.00000000\nv 10.00000000 0.00000000 5.00000000 0.00000000 1.00000000 0.00000000\nv 10.00000000 0.00000000 10.00000000 0.00000000 1.00000000 0.00000000\nv 10.00000000 5.00000000 5.00000000 0.00000000 1.00000000 0.00000000\nv 10.00000000 5.00000000 10.00000000 0.00000000 1.00000000 0.00000000\nf 2 4 1\nf 5 2 1\nf 1 4 3\nf 3 5 1\nf 2 8 4\nf 6 2 5\nf 6 8 2\nf 4 8 3\nf 7 5 3\nf 3 8 7\nf 7 6 5\nf 8 6 7'

In [ ]:

tfs = np.eye(4)
# First is small translation to get the anchor to front left corner
# Then translate anchor to desired location
# tfs[:3,3] = np.array([5,10,-2.5]) + np.array([0, 0, 10])
tfs[:3,3] = np.array([-40.0, -30.0, -2.5]) + np.array([0, 0, 10])

sm = trimesh.creation.box(np.array([100.0, 100.0, 5.0]), transform=tfs)
sm.visual.vertex_colors = [0.0, 1.0, 0.0, 0.1]

low_bounds = [-1, -1, -1]
up_bounds = [15, 20, 15]
increment = 0.1

x = np.arange(low_bounds[0], up_bounds[0], increment)
y = np.arange(low_bounds[1], up_bounds[1], increment)
z = np.arange(low_bounds[2], up_bounds[2], increment)

sdf_points = []

for xi in x:
    for yi in y:
        for zi in z:
            sdf_points.append([xi, yi, zi])

sdf_points = np.array(sdf_points)


In [34]:
sdf = mesh_to_sdf(sm, sdf_points)
print(sdf.shape)

(5376000,)


In [3]:
colors = np.zeros(sdf_points.shape)
colors[sdf < 0, 2] = 1
colors[sdf > 0, 0] = 1
cloud = pyrender.Mesh.from_points(sdf_points, colors=colors)
scene = pyrender.Scene()
scene.add(cloud)
viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2, show_world_axis=True)

In [45]:
sdf_new = np.reshape(sdf, (x.shape[0],y.shape[0],z.shape[0]))
print(sdf_new.shape)
np.save("box_sdf.npy", sdf_new)


(160, 210, 160)


In [50]:
# This gives the volume normals
sdf_grad = np.array(np.gradient(sdf_new, x, y, z)) 
sdf_grad = np.moveaxis(sdf_grad, 0, -1)
print(sdf_grad.shape)
np.save("box_sdf_gradient.npy", sdf_grad)


(160, 210, 160, 3)


In [54]:
sdf_laplacian = np.zeros((*sdf_grad.shape, 3))
for i in range(3):
    sdf_grad_ax = sdf_grad[:,:,:,i]
    sdf_grad_ax_grad = np.array(np.gradient(sdf_grad_ax, x, y, z))
    sdf_grad_ax_grad = np.moveaxis(sdf_grad_ax_grad, 0, -1)
    sdf_laplacian[:, :, :, i, :] = sdf_grad_ax_grad
sdf_laplacian = np.swapaxes(sdf_laplacian, 3, 4)

In [56]:
print(sdf_laplacian.shape)
np.save("box_sdf_laplacian.npy", sdf_laplacian)

(160, 210, 160, 3, 3)


In [25]:
def find_nearest_ind(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def compute_ind(low_bound, up_bound, inc, v):
    if v <= low_bound:
        return 0
    elif v >= up_bound:
        return -1
    else:
        shifted_v = v - low_bound
        return np.floor(shifted_v / inc).astype(int)

def get_inds(x, y, z, p):
    inds = [0, 0, 0]
    # inds[0] = find_nearest_ind(x,p[0])
    # inds[1] = find_nearest_ind(y,p[1])
    # inds[2] = find_nearest_ind(z,p[2])
    inds[0] = compute_ind(low_bounds[0], up_bounds[0], increment, p[0])
    inds[1] = compute_ind(low_bounds[1], up_bounds[1], increment, p[1])
    inds[2] = compute_ind(low_bounds[2], up_bounds[2], increment, p[2])
    inds = np.array(inds)
    return inds

def get_val(x, y, z, sdf, p):
    inds = get_inds(x, y, z, p)
    val = sdf[inds[0], inds[1], inds[2]]
    return val

def get_grads(x, y, z, sdf_grad, p):
    ind = get_inds(x, y, z, p)
    grad = sdf_grad[ind[0], ind[1], ind[2], :]
    return grad


In [39]:
# Check some SD values and normals

# Exterior
p = [12, 10, 7.5]
print(get_val(x, y, z, sdf_new, p))
print(get_grads(x, y, z, sdf_grad, p))

# Interior
p = [2, 5, 9]
print(get_val(x, y, z, sdf_new, p))
print(get_grads(x, y, z, sdf_grad, p))

# Edge
p = [10, 1, 7.5]
print(get_val(x, y, z, sdf_new, p))
print(get_grads(x, y, z, sdf_grad, p))


1.9834445714950562
[0.99361539 0.04580736 0.02280772]
-1.0142085552215576
[ 0.07554114 -0.04846454  0.99161118]
-0.13682080805301666
[ 1.6595573  -0.09490989  0.37633255]


In [27]:
def collision_correction(p, x, y, z, sdf, sdf_grad):
    correction = np.array([0.0, 0.0, 0.0])
    sdf_val = get_val(x, y, z, sdf, p)
    normal = get_grads(x, y, z, sdf_grad, p)
    # Negative SDF value means inside the solid
    if sdf_val <= 0:
        correction = -1 * sdf_val * normal
    return correction

In [40]:
# Check some correction factors

# Exterior
p = [12, 10, 7.5]
print(collision_correction(p, x, y, z, sdf_new, sdf_grad))

# Interior
p = [2, 5, 9]
print(collision_correction(p, x, y, z, sdf_new, sdf_grad))
# Edge
p = [10, 1, 7.5]
print(collision_correction(p, x, y, z, sdf_new, sdf_grad))


[0. 0. 0.]
[ 0.07661447 -0.04915315  1.00570054]
[ 0.22706197 -0.01298565  0.05149012]


In [29]:
def get_laplacians(x, y, z, sdf_laplacian, p):
    ind = get_inds(x, y, z, p)
    laplacian = sdf_laplacian[ind[0], ind[1], ind[2], :, :]
    return laplacian

In [41]:
# Check some Laplacians
# For box shape, we don't expect the normal to change much other than at skeleton points

# Exterior
p = [12, 10, 7.5]
print(get_val(x, y, z, sdf_new, p))
print(get_grads(x, y, z, sdf_grad, p))
print(get_laplacians(x, y, z, sdf_laplacian, p))

# Interior
p = [2, 5, 9]
print(get_val(x, y, z, sdf_new, p))
print(get_grads(x, y, z, sdf_grad, p))
print(get_laplacians(x, y, z, sdf_laplacian, p))

# Edge
p = [10, 1, 7.5]
print(get_val(x, y, z, sdf_new, p))
print(get_grads(x, y, z, sdf_grad, p))
print(get_laplacians(x, y, z, sdf_laplacian, p))

1.9834445714950562
[0.99361539 0.04580736 0.02280772]
[[ 0.0064671  -0.03248751 -0.00202656]
 [-0.03248751  0.06632209  0.43266714]
 [-0.00202656  0.43266714 -0.49413145]]
-1.0142085552215576
[ 0.07554114 -0.04846454  0.99161118]
[[-0.19091964 -0.13198555  0.00546575]
 [-0.13198555  0.01778007  0.02129674]
 [ 0.00546575  0.02129674 -0.01696348]]
-0.13682080805301666
[ 1.6595573  -0.09490989  0.37633255]
[[ 6.66541383  0.66915974 -2.20883097]
 [ 0.66915974  2.4477548  -3.36828227]
 [-2.20883097 -3.36828227  2.28267405]]
